# Agent에 도구 추가

In [ ]:
# API-KEY 읽어오기

In [ ]:
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

### 도구 정의

In [ ]:
# pip install langchain-tavily

In [ ]:
def multiply(a: int, b: int) -> int:
# 웹 검색을 도구
# 도구 리스트
# 도구 테스트
# 도구 테스트

In [ ]:
# LLM에 입력 되는 값 확인

## ReAct Agent를 수동으로 구현

- `create_react_agent`를 사용하지 않고 수동 구현

```
chatbot (LLM 추론) → route_tools (판단) → tools (도구 실행) → chatbot (결과 처리)
                   ↘ END (완료)
```

각 단계를 더 정교하게 제어하고 커스터마이징할 수 있다.

In [ ]:
# 에이전트의 상태 정의 클래스
class State(TypedDict):
# 상태 기반 워크플로우 생성
# LLM이 도구 호출 여부 판단
# chatbot 노드 함수
def chatbot(state: State):
# 워크플로우에 chatbot 노드 추가

### 도구 실행 함수 생성

#### ToolMessage
- LangGraph에서 도구(tool) 실행 결과를 표현하기 위한 특별한 메시지 객체.  
- 일반 AIMessage나 HumanMessage처럼 대화의 한 턴을 나타내지만, 특히 “이건 도구 실행의 결과다”라는 것을 그래프 런타임에 알려주는 역할.

In [ ]:
# 도구 실행 클래스
class BasicToolNode:
    def __init__(self, tools: list) -> None:
    def __call__(self, inputs: State):
        # 입력 상태에서 메시지 리스트를 추출.
            # 도구 이름을 이용해 도구 호출
            # 도구 결과를 `ToolMessage` 형식으로 변환하여 출력 리스트에 추가
        # 도구 호출 결과를 포함한 메시지 리스트를 반환
# BasicToolNode 인스턴스 생성
# tools 노드를 워크플로우에 추가

### 조건부 에지(conditional_edge) 추가

In [ ]:
def route_tools(state: State):
    # LLM 응답에 `tool_calls`가 있으면 `tools` 노드로 이동.
    # 없으면 END 노드로 이동 (종료)
# 조건부 라우팅: tools로 이동하거나 END로 종료.
    # 라우팅 결과를 노드 이름으로 매핑
# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
# 워크플로우 시작점에서 chatbot 노드로 이동
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성

### graph.invoke vs. graph.stream

--------

## create_react_agent 로 ReAct Agent 구현

In [ ]:
# ReAct 에이전트 생성

(대화 예시)
```
사용자:  지금 서울의 날씨가 어때?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_LBshVOFPi8aaOY4OMvQsg2xo)
 Call ID: call_LBshVOFPi8aaOY4OMvQsg2xo

 사용자:  100 곱하기 2 는 얼마야?
================================== Ai Message ==================================
Tool Calls:
  multiply (call_2E2bRfRICTDKAurlV8S6i7dm)
```

In [ ]:
def stream_graph_updates(user_input: str):
    # 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
            # 이벤트의 마지막 메시지 출력
            # ToolMessage는 출력 제외
            # if isinstance(last_msg, ToolMessage):
            #     continue
        # 워크플로우 실행

----------------------
우리는 `messages`를 상태로 사용하고, **도구가 바인딩된 채팅 모델**을 사용하는 그래프를 만들었습니다.

이 그래프는 다음과 같은 동작을 할 수 있었습니다:  
* 도구 호출을 반환
* 자연어 응답을 반환

# 라우터 (Router)

1. **도구를 실제로 호출하는 노드**를 추가합니다.
2. **조건부 엣지**를 추가하여, 채팅 모델의 출력을 확인한 뒤

   * 도구 호출이 있다면 도구 노드로
   * 도구 호출이 없다면 바로 종료 노드로 이동하도록 라우팅합니다.

![라우터 예시 이미지](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac6543c3d4df239a4ed1_router1.png)

In [ ]:
def multiply(a: int, b: int) -> int:

우리는 [내장된 `ToolNode`](https://langchain-ai.github.io/langgraph/reference/prebuilt/?h=tools+condition#toolnode)를 사용하며,
**초기화 시 도구 목록(list of tools)** 만 전달하면 됩니다.

또한, [내장된 `tools_condition`](https://langchain-ai.github.io/langgraph/reference/prebuilt/?h=tools+condition#tools_condition)를
**조건부 엣지(conditional edge)** 로 사용합니다.

In [ ]:
# Node
def tool_calling_llm(state: MessagesState):
# Build graph
    # 만약 assistant의 마지막 메시지(결과)가 도구 호출이면 -> tools_condition은 tools 노드로 이동합니다.
    # 만약 assistant의 마지막 메시지가 도구 호출이 아니면 -> tools_condition은 END 노드로 이동합니다.